# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120542e+02     1.603669e+00
 * time: 0.7700278759002686
     1     1.046274e+01     8.938910e-01
 * time: 2.1400129795074463
     2    -1.139931e+01     9.560629e-01
 * time: 2.7520320415496826
     3    -3.401978e+01     6.885031e-01
 * time: 3.6644508838653564
     4    -4.730279e+01     5.317986e-01
 * time: 4.5343358516693115
     5    -5.678136e+01     2.246459e-01
 * time: 5.354042053222656
     6    -5.975356e+01     1.609857e-01
 * time: 5.961169004440308
     7    -6.089889e+01     6.173704e-02
 * time: 6.570036888122559
     8    -6.136184e+01     5.009785e-02
 * time: 7.143466949462891
     9    -6.160222e+01     6.230059e-02
 * time: 7.711845874786377
    10    -6.180005e+01     2.804718e-02
 * time: 8.301197052001953
    11    -6.194512e+01     2.444181e-02
 * time: 8.890339851379395
    12    -6.203490e+01     1.992398e-02
 * time: 9.459983825683594
    13    -6.209368e+01     1.493984e-02
 * time: 10.013208866119385
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671071
    AtomicLocal         -18.8557723
    AtomicNonlocal      14.8522682
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485380 
    Xc                  -19.3336823

    total               -62.261666455216